In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [2]:
# test = pd.read_csv('/kaggle/input/isic-2024-challenge/test-metadata.csv')
# train = pd.read_csv('/kaggle/input/isic-2024-challenge/train-metadata.csv')
path = '/kaggle/input/isic-2024-challenge/'

In [3]:
import warnings
warnings.filterwarnings("ignore")
import keras
import json
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import models
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from tqdm import tqdm
from tensorflow.keras import backend as K
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from torchvision import transforms
import copy
import logging as log
import random
import h5py
import os
import io
from PIL import Image

2024-09-09 14:02:42.716114: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-09 14:02:42.716222: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-09 14:02:42.857789: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
def align_columns(train_df, test_df):
    # Get the columns that are in test_df but not in train_df
    extra_columns = set(test_df.columns) - set(train_df.columns)
    
    # Drop the extra columns from test_df
    test_df.drop(columns=extra_columns, inplace=True)
    
    # Ensure both DataFrames have the same columns in the same order, excluding 'target'
    common_columns = [col for col in train_df.columns if col != 'target']
    test_df = test_df[common_columns]
    
    return train_df, test_df

# Example usage
# train_df, test_df = align_columns(train_df, test_df)

import numpy as np

def create_features(train_df, test_df):
    # Define a small constant to avoid division by zero
    epsilon = 1e-6

    # Check and create 'lesion_size_ratio' if columns exist
    if 'clin_size_long_diam_mm' in train_df.columns and 'tbp_lv_minorAxisMM' in train_df.columns:
        train_df['lesion_size_ratio'] = train_df['clin_size_long_diam_mm'] / (train_df['tbp_lv_minorAxisMM'] + epsilon)
    if 'clin_size_long_diam_mm' in test_df.columns and 'tbp_lv_minorAxisMM' in test_df.columns:
        test_df['lesion_size_ratio'] = test_df['clin_size_long_diam_mm'] / (test_df['tbp_lv_minorAxisMM'] + epsilon)

    # Check and create 'perimeter_area_ratio' if columns exist
    if 'tbp_lv_perimeterMM' in train_df.columns and 'tbp_lv_areaMM2' in train_df.columns:
        train_df['perimeter_area_ratio'] = train_df['tbp_lv_perimeterMM'] / (train_df['tbp_lv_areaMM2'] + epsilon)
    if 'tbp_lv_perimeterMM' in test_df.columns and 'tbp_lv_areaMM2' in test_df.columns:
        test_df['perimeter_area_ratio'] = test_df['tbp_lv_perimeterMM'] / (test_df['tbp_lv_areaMM2'] + epsilon)

    # Check and create 'color_contrast_AB' if columns exist
    if 'tbp_lv_deltaA' in train_df.columns and 'tbp_lv_deltaB' in train_df.columns:
        train_df['color_contrast_AB'] = np.sqrt(train_df['tbp_lv_deltaA']**2 + train_df['tbp_lv_deltaB']**2)
    if 'tbp_lv_deltaA' in test_df.columns and 'tbp_lv_deltaB' in test_df.columns:
        test_df['color_contrast_AB'] = np.sqrt(test_df['tbp_lv_deltaA']**2 + test_df['tbp_lv_deltaB']**2)

    # Check and create 'color_contrast_LB' if columns exist
    if 'tbp_lv_deltaL' in train_df.columns and 'tbp_lv_deltaLB' in train_df.columns:
        train_df['color_contrast_LB'] = np.sqrt(train_df['tbp_lv_deltaL']**2 + train_df['tbp_lv_deltaLB']**2)
    if 'tbp_lv_deltaL' in test_df.columns and 'tbp_lv_deltaLB' in test_df.columns:
        test_df['color_contrast_LB'] = np.sqrt(test_df['tbp_lv_deltaL']**2 + test_df['tbp_lv_deltaLB']**2)

    # Check and create 'symmetry_asymmetry_ratio' if columns exist
    if 'tbp_lv_symm_2axis' in train_df.columns and 'tbp_lv_symm_2axis_angle' in train_df.columns:
        train_df['symmetry_asymmetry_ratio'] = train_df['tbp_lv_symm_2axis'] / (train_df['tbp_lv_symm_2axis_angle'] + epsilon)
    if 'tbp_lv_symm_2axis' in test_df.columns and 'tbp_lv_symm_2axis_angle' in test_df.columns:
        test_df['symmetry_asymmetry_ratio'] = test_df['tbp_lv_symm_2axis'] / (test_df['tbp_lv_symm_2axis_angle'] + epsilon)

    columns_to_drop = [
        'tbp_lv_deltaA', 'tbp_lv_deltaB', 'tbp_lv_deltaL', 'tbp_lv_deltaLB',
        'tbp_lv_symm_2axis', 'tbp_lv_symm_2axis_angle', 'clin_size_long_diam_mm',
        'tbp_lv_minorAxisMM', 'tbp_lv_perimeterMM', 'tbp_lv_areaMM2'
    ]

    train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns], inplace=True)
    test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns], inplace=True)

    return train_df, test_df



In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler

def load_data(train_path, test_path):
    train_df = pd.read_csv(train_path)
    test_df = pd.read_csv(test_path)
    return train_df, test_df

def select_columns(train_df, test_df, trn_selected, test_selected):
    train_df = train_df[trn_selected]
    test_df = test_df[test_selected]
    return train_df, test_df

def fill_missing_values(df):
    for column in df.columns:
        if df[column].isnull().any():
            df[column] = df[column].fillna(df[column].mode()[0])
    return df

def create_features(train_df, test_df):
    epsilon = 1e-6

    try:
        if 'clin_size_long_diam_mm' in train_df.columns and 'tbp_lv_minorAxisMM' in train_df.columns:
            train_df['lesion_size_ratio'] = train_df['clin_size_long_diam_mm'] / (train_df['tbp_lv_minorAxisMM'] + epsilon)
        if 'clin_size_long_diam_mm' in test_df.columns and 'tbp_lv_minorAxisMM' in test_df.columns:
            test_df['lesion_size_ratio'] = test_df['clin_size_long_diam_mm'] / (test_df['tbp_lv_minorAxisMM'] + epsilon)

        if 'tbp_lv_perimeterMM' in train_df.columns and 'tbp_lv_areaMM2' in train_df.columns:
            train_df['perimeter_area_ratio'] = train_df['tbp_lv_perimeterMM'] / (train_df['tbp_lv_areaMM2'] + epsilon)
        if 'tbp_lv_perimeterMM' in test_df.columns and 'tbp_lv_areaMM2' in test_df.columns:
            test_df['perimeter_area_ratio'] = test_df['tbp_lv_perimeterMM'] / (test_df['tbp_lv_areaMM2'] + epsilon)

        if 'tbp_lv_deltaA' in train_df.columns and 'tbp_lv_deltaB' in train_df.columns:
            train_df['color_contrast_AB'] = np.sqrt(train_df['tbp_lv_deltaA']**2 + train_df['tbp_lv_deltaB']**2)
        if 'tbp_lv_deltaA' in test_df.columns and 'tbp_lv_deltaB' in test_df.columns:
            test_df['color_contrast_AB'] = np.sqrt(test_df['tbp_lv_deltaA']**2 + test_df['tbp_lv_deltaB']**2)

        if 'tbp_lv_deltaL' in train_df.columns and 'tbp_lv_deltaLB' in train_df.columns:
            train_df['color_contrast_LB'] = np.sqrt(train_df['tbp_lv_deltaL']**2 + train_df['tbp_lv_deltaLB']**2)
        if 'tbp_lv_deltaL' in test_df.columns and 'tbp_lv_deltaLB' in test_df.columns:
            test_df['color_contrast_LB'] = np.sqrt(test_df['tbp_lv_deltaL']**2 + test_df['tbp_lv_deltaLB']**2)

        if 'tbp_lv_symm_2axis' in train_df.columns and 'tbp_lv_symm_2axis_angle' in train_df.columns:
            train_df['symmetry_asymmetry_ratio'] = train_df['tbp_lv_symm_2axis'] / (train_df['tbp_lv_symm_2axis_angle'] + epsilon)
        if 'tbp_lv_symm_2axis' in test_df.columns and 'tbp_lv_symm_2axis_angle' in test_df.columns:
            test_df['symmetry_asymmetry_ratio'] = test_df['tbp_lv_symm_2axis'] / (test_df['tbp_lv_symm_2axis_angle'] + epsilon)

        columns_to_drop = [
            'tbp_lv_deltaA', 'tbp_lv_deltaB', 'tbp_lv_deltaL', 'tbp_lv_deltaLB',
            'tbp_lv_symm_2axis', 'tbp_lv_symm_2axis_angle', 'clin_size_long_diam_mm',
            'tbp_lv_minorAxisMM', 'tbp_lv_perimeterMM', 'tbp_lv_areaMM2'
        ]

        train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns], inplace=True)
        test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns], inplace=True)

    except Exception as e:
        print(f"Error in create_features: {e}")

    return train_df, test_df




def encode_and_scale(train_df, test_df, categorical_cols, numerical_cols):
    try:
#         encoder = OneHotEncoder(drop='first', sparse=False)
        encoder = OneHotEncoder(drop='first', sparse_output=False)
        encoded_train = pd.DataFrame(encoder.fit_transform(train_df[categorical_cols]), 
                                     columns=encoder.get_feature_names_out(categorical_cols))
        encoded_test = pd.DataFrame(encoder.transform(test_df[categorical_cols]), 
                                    columns=encoder.get_feature_names_out(categorical_cols))

        train_df = pd.concat([train_df.drop(columns=categorical_cols), encoded_train], axis=1)
        test_df = pd.concat([test_df.drop(columns=categorical_cols), encoded_test], axis=1)

        scaler = StandardScaler()
        train_df[numerical_cols] = scaler.fit_transform(train_df[numerical_cols])
        test_df[numerical_cols] = scaler.transform(test_df[numerical_cols])

        train_df = train_df.astype({col: 'float32' for col in train_df.select_dtypes(include='float64').columns})
        test_df = test_df.astype({col: 'float32' for col in test_df.select_dtypes(include='float64').columns})

    except Exception as e:
        print(f"Error in encode_and_scale: {e}")

    return train_df, test_df

def main(train_path, test_path):
    trn_selected = ['isic_id', 'target', 'patient_id', 'age_approx', 'sex', 'anatom_site_general',
                    'clin_size_long_diam_mm', 'image_type', 'tbp_tile_type', 'tbp_lv_A', 'tbp_lv_Aext', 
                    'tbp_lv_B', 'tbp_lv_Bext', 'tbp_lv_C', 'tbp_lv_Cext', 'tbp_lv_H', 'tbp_lv_Hext', 
                    'tbp_lv_L', 'tbp_lv_Lext', 'tbp_lv_areaMM2', 'tbp_lv_area_perim_ratio', 
                    'tbp_lv_color_std_mean', 'tbp_lv_deltaA', 'tbp_lv_deltaB', 'tbp_lv_deltaL', 
                    'tbp_lv_deltaLB', 'tbp_lv_deltaLBnorm', 'tbp_lv_eccentricity', 'tbp_lv_location', 
                    'tbp_lv_location_simple', 'tbp_lv_minorAxisMM', 'tbp_lv_nevi_confidence', 
                    'tbp_lv_norm_border', 'tbp_lv_norm_color', 'tbp_lv_perimeterMM', 
                    'tbp_lv_radial_color_std_max', 'tbp_lv_stdL', 'tbp_lv_stdLExt', 'tbp_lv_symm_2axis', 
                    'tbp_lv_symm_2axis_angle', 'tbp_lv_x', 'tbp_lv_y', 'tbp_lv_z', 'attribution', 
                    'copyright_license']

    test_selected = ['isic_id', 'patient_id', 'age_approx', 'sex', 'anatom_site_general',
                     'clin_size_long_diam_mm', 'image_type', 'tbp_tile_type', 'tbp_lv_A', 'tbp_lv_Aext', 
                     'tbp_lv_B', 'tbp_lv_Bext', 'tbp_lv_C', 'tbp_lv_Cext', 'tbp_lv_H', 'tbp_lv_Hext', 
                     'tbp_lv_L', 'tbp_lv_Lext', 'tbp_lv_areaMM2', 'tbp_lv_area_perim_ratio', 
                     'tbp_lv_color_std_mean', 'tbp_lv_deltaA', 'tbp_lv_deltaB', 'tbp_lv_deltaL', 
                     'tbp_lv_deltaLB', 'tbp_lv_deltaLBnorm', 'tbp_lv_eccentricity', 'tbp_lv_location', 
                     'tbp_lv_location_simple', 'tbp_lv_minorAxisMM', 'tbp_lv_nevi_confidence', 
                     'tbp_lv_norm_border', 'tbp_lv_norm_color', 'tbp_lv_perimeterMM', 
                     'tbp_lv_radial_color_std_max', 'tbp_lv_stdL', 'tbp_lv_stdLExt', 'tbp_lv_symm_2axis', 
                     'tbp_lv_symm_2axis_angle', 'tbp_lv_x', 'tbp_lv_y', 'tbp_lv_z', 'attribution', 
                     'copyright_license']

    categorical_cols = ['sex', 'anatom_site_general', 'image_type', 'tbp_tile_type', 
                        'tbp_lv_location', 'tbp_lv_location_simple']

    numerical_cols = ['age_approx', 'tbp_lv_A', 'tbp_lv_Aext', 'tbp_lv_B', 'tbp_lv_Bext',
                      'tbp_lv_C', 'tbp_lv_Cext', 'tbp_lv_H', 'tbp_lv_Hext', 'tbp_lv_L',
                      'tbp_lv_Lext', 'tbp_lv_area_perim_ratio', 'tbp_lv_color_std_mean',
                      'tbp_lv_deltaLBnorm', 'tbp_lv_eccentricity', 'tbp_lv_nevi_confidence',
                      'tbp_lv_norm_border', 'tbp_lv_norm_color', 'tbp_lv_radial_color_std_max',
                      'tbp_lv_stdL', 'tbp_lv_stdLExt', 'tbp_lv_x', 'tbp_lv_y', 'tbp_lv_z',
                      'lesion_size_ratio', 'perimeter_area_ratio', 'color_contrast_AB',
                      'color_contrast_LB', 'symmetry_asymmetry_ratio']

    train_df, test_df = load_data(train_path, test_path)
    train_df, test_df = select_columns(train_df, test_df, trn_selected, test_selected)
    train_df = fill_missing_values(train_df)
    test_df = fill_missing_values(test_df)
    train_df, test_df = create_features(train_df, test_df)
    train_df, test_df = encode_and_scale(train_df, test_df, categorical_cols, numerical_cols)

    train_df.drop(['attribution', 'copyright_license'], axis=1, inplace=True)
    test_df.drop(['attribution', 'copyright_license'], axis=1, inplace=True)

    return train_df, test_df

# Example usage
train_path = '/kaggle/input/isic-2024-challenge/train-metadata.csv'
test_path = '/kaggle/input/isic-2024-challenge/test-metadata.csv'
train_df, test_df = main(train_path, test_path)


In [6]:
# train_df

In [7]:
from collections import namedtuple

CandidateInfoTuple = namedtuple(
    'CandidateInfoTuple',
    'isNodule_bool, isic_id, patient_id, attributes',
)


def getCandidateInfoList(df, is_train=True, requireOnDisk_bool=True):
    candidateInfo_list = []

    # Define columns to drop based on whether it's train or test DataFrame
    if is_train:
        columns_to_drop = ['isic_id', 'target', 'patient_id']
    else:
        columns_to_drop = ['isic_id', 'patient_id']

    # Iterate over each row in the dataframe
    for _, row in df.iterrows():
        # Drop unnecessary columns to get attributes
        attributes = row.drop(columns_to_drop).tolist()

        # Wrap each attribute in a single-element list
#         attributes = [[attr] for attr in attributes]

        # Determine the value of isNodule_bool, if 'target' exists
        isNodule_bool = row['target'] if 'target' in df.columns else None

        # Append the CandidateInfoTuple to the list
        candidateInfo_list.append(CandidateInfoTuple(
            isNodule_bool=isNodule_bool,
            isic_id=row['isic_id'],
            patient_id=row['patient_id'],
            attributes=attributes,
        ))

    # Sort the candidate info list in reverse order
    candidateInfo_list.sort(reverse=True)
    return candidateInfo_list



# Usage example:
# metadata_df = pd.read_csv('train_metadata.csv')
# candidates = getCandidateInfoList()
# print(candidates[0])  # Modify as needed

In [8]:
class Img:
    def __init__(self, isic_id, img_path, transform=None):
        with h5py.File(img_path, 'r') as f:
            try:
                dset = f[isic_id]
                self.image_data = dset[()]
                image_data_io = io.BytesIO(self.image_data)
                self.image = Image.open(image_data_io)

                # Apply transform if provided
                if transform:
                    self.image = transform(self.image)
                    self.image_array = self.image.numpy()  # Convert the transformed tensor to a NumPy array

                    # The transform will already convert the image to (C, H, W) when using ToTensor
                    # No need to transpose here if ToTensor was applied
                else:
                    # No transform; handle the image normally
                    self.image_array = np.array(self.image)
                    if self.image_array.dtype == np.uint8:
                        self.image_array = self.image_array / 255.0  # Normalize for display

                    # Transpose from (H, W, C) to (C, H, W) if necessary
                    if self.image_array.shape[-1] == 3:  # check if last dimension is channels
                        self.image_array = np.transpose(self.image_array, (2, 0, 1))

#                 print(f"Final shape: {self.image_array.shape}")

            except KeyError:
                raise ValueError(f"ISIC ID '{isic_id}' not found in 'train-image.hdf5'")


In [9]:
# /kaggle/input/isic-skin-dset
# def load_npy(isic_id, img_path):
#     return np.load(os.path.join(img_path, f"{isic_id}.npy"), mmap_mode='r')


def load_npy(isic_id, img_path):
    """
    Load the .npy file corresponding to the given ISIC ID.

    Args:
        isic_id (str): The ISIC ID of the image.
        img_path (str): The directory path where the .npy files are stored.

    Returns:
        np.ndarray: The loaded image data.
    """
    file_path = os.path.join(img_path, f"{isic_id}.npy")
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    return np.load(file_path)

In [10]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from PIL import Image

class ImgDatasetTF:
    def __init__(self,
                 val_stride=0,
                 isValSet_bool=None,
                 isic_id=None,
                 img_path=None,
                 sortby_str='random',
                 ratio_int=0,
                 is_train=None,
                 train_df=None,
                 transform=None,
                 chunksize=10000):
        self.is_train = is_train
        self.train_df = train_df
        self.img_path = img_path
        self.transform = transform
        self.chunksize = chunksize
        self.ratio_int = ratio_int
        self.imgInfo_list = []

        # Process the DataFrame in chunks
        for start in range(0, len(self.train_df), self.chunksize):
            chunk = train_df.iloc[start:start + self.chunksize]
            self.imgInfo_list.extend(getCandidateInfoList(chunk, self.is_train))

        # Filter by isic_id if provided
        if isic_id:
            self.imgInfo_list = [x for x in self.imgInfo_list if x.isic_id == isic_id]

        # Validation set handling
        if isValSet_bool:
            assert val_stride > 0, val_stride
            self.imgInfo_list = self.imgInfo_list[::val_stride]
            assert self.imgInfo_list
        elif val_stride > 0:
            del self.imgInfo_list[::val_stride]
            assert self.imgInfo_list

        # Sort by the specified method
        if sortby_str == 'random':
            random.shuffle(self.imgInfo_list)

        # Split into positive and negative lists if ratio is specified
        self.negative_list = [nt for nt in self.imgInfo_list if not nt.isNodule_bool]
        self.pos_list = [nt for nt in self.imgInfo_list if nt.isNodule_bool]
        
        # Automatically determine number of attributes from the first sample
        if self.imgInfo_list:
            self.num_attributes = len(self.imgInfo_list[0].attributes)
        else:
            self.num_attributes = 62 
        
    def shuffleSamples(self):
        if self.ratio_int:
            random.shuffle(self.negative_list)
            random.shuffle(self.pos_list)

    def __len__(self):
        if self.ratio_int:
            return 200000  # Or some other large number if balancing ratio
        return len(self.imgInfo_list)
    
    def to_tf_dataset(self, batch_size=32):
        def generator():
            for idx in range(len(self)):
                attributes, isic_id, patient_id, labels = self[idx]
                yield attributes, isic_id, patient_id, labels

        # Update the output_signature to include all 4 values
        dataset = tf.data.Dataset.from_generator(generator, 
                                                 output_signature=(
                                                     tf.TensorSpec(shape=(self.num_attributes,), dtype=tf.float32),  # attributes
                                                     tf.TensorSpec(shape=(), dtype=tf.string),  # isic_id
                                                     tf.TensorSpec(shape=(), dtype=tf.string),  # patient_id
                                                     tf.TensorSpec(shape=(2,), dtype=tf.float32)  # labels
                                                 ))
        dataset = dataset.batch(batch_size)
        dataset = dataset.prefetch(tf.data.AUTOTUNE)
        return dataset

    def __getitem__(self, ndx):
        # Handle ratio balancing between positive and negative samples
        if self.ratio_int:
            pos_ndx = ndx // (self.ratio_int + 1)
            if ndx % (self.ratio_int + 1):
                neg_ndx = ndx - 1 - pos_ndx
                neg_ndx %= len(self.negative_list)
                imgInfo_tup = self.negative_list[neg_ndx]
            else:
                pos_ndx %= len(self.pos_list)
                imgInfo_tup = self.pos_list[pos_ndx]
        else:
            imgInfo_tup = self.imgInfo_list[ndx]

        # Load the image using the ISIC ID on-the-fly
#         img_t = load_npy(imgInfo_tup.isic_id, self.img_path)

#         # Convert the NumPy array to a PIL Image
#         img_t = Image.fromarray(img_t)

#         # Apply transformations if provided
#         if self.transform:
#             img_t = self.transform(img_t)

#         # Convert image to TensorFlow tensor
#         img_t = tf.convert_to_tensor(np.array(img_t), dtype=tf.float32)

        # Convert attributes to a NumPy array
        # Convert attributes to a NumPy array and reshape to (62,)
        attributes = np.array(imgInfo_tup.attributes, dtype=np.float32).reshape(-1)
#         label = imgInfo_tup.label if hasattr(imgInfo_tup, 'label') else None
        labels = np.array([1, 0], dtype=np.float32) if imgInfo_tup.isNodule_bool else np.array([0, 1], dtype=np.float32)
    
        return attributes, imgInfo_tup.isic_id, imgInfo_tup.patient_id, labels

#         return img_t, imgInfo_tup.isic_id, imgInfo_tup.patient_id, attributes, labels



# Define transformations for training (no resize, just random flip)
def transform_train(img):
    img = tf.image.random_flip_left_right(img)  # Randomly flip horizontally
    img = tf.cast(img, tf.float32)  # Cast to float32
    img = img / 255.0  # Normalize to [0, 1]
    return img


# Define transformations for testing (include resize)
def transform_test(img):
    img = tf.image.resize(img, [48, 48])  # Resize to 48x48
    img = tf.cast(img, tf.float32)  # Cast to float32
#     img = img / 255.0  # Normalize to [0, 1]
    return img


# Load an example image from the dataset and display it
def display_image_from_dataset(img_dataset, index):
    img_t, isic_id, patient_id, attributes = img_dataset[index]
    
    # Convert the TensorFlow tensor back to NumPy for visualization
    img_np = img_t.numpy()

    # Plot the image using matplotlib
    plt.imshow(img_np)
    plt.title(f"ISIC ID: {isic_id}, Patient ID: {patient_id}")
    plt.axis('off')
    plt.show()


# Example usage:
# Initialize the dataset (use test_df and img_path from your environment)
# img_dataset_train = ImgDatasetTF(transform=transform_train, train_df=train_df, img_path="/kaggle/input/training/npy_images")

# Display one image at index 0
# display_image_from_dataset(img_dataset_train, index=0)


In [11]:
import tensorflow as tf
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import numpy as np
from tqdm import tqdm

# Function to calculate metrics
def calculate_metrics(true_labels, predicted, num_correct_neg, total_neg, num_correct_pos, total_pos):
    if len(true_labels.shape) > 1:
        true_labels = np.argmax(true_labels, axis=1)
    if len(predicted.shape) > 1:
        predicted = np.argmax(predicted, axis=1)

    cm = confusion_matrix(true_labels, predicted)
    
    if cm.shape == (2, 2):
        tn, fp, fn, tp = cm.ravel()
    else:
        tn = fp = fn = tp = 0

    precision = precision_score(true_labels, predicted, zero_division=0)
    recall = recall_score(true_labels, predicted, zero_division=0)
    f1 = f1_score(true_labels, predicted, zero_division=0)

    acc = 100.0 * (num_correct_neg + num_correct_pos) / (total_neg + total_pos)

    print(f'TN={tn}, FP={fp}, FN={fn}, TP={tp}, Precision={precision:.2f}, Recall={recall:.2f}, F1={f1:.2f}')
    print(f'Accuracy: {acc:.2f}% ({num_correct_neg} of {total_neg} negative, {num_correct_pos} of {total_pos} positive)')

# Training and validation function with early stopping
def train_model(model, train_dataset, val_dataset, epochs, optimizer, loss_fn, patience=3):
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)
    best_val_loss = float('inf')
    patience_counter = 0

    for epoch in range(epochs):
        # Training phase
        running_loss = 0.0
        num_correct_neg = num_correct_pos = total_neg = total_pos = 0
        num_samples = 0

        for attributes, isic_id, patient_id, labels in tqdm(train_dataset):
            with tf.GradientTape() as tape:
                outputs = model(attributes, training=True)
                loss = loss_fn(labels, outputs)
                running_loss += loss.numpy()

            grads = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

            predicted = tf.argmax(outputs, axis=1)
            true_labels = tf.argmax(labels, axis=1)

            correct_neg = tf.reduce_sum(tf.cast((predicted == 0) & (true_labels == 0), tf.int32)).numpy()
            correct_pos = tf.reduce_sum(tf.cast((predicted == 1) & (true_labels == 1), tf.int32)).numpy()
            num_correct_neg += correct_neg
            num_correct_pos += correct_pos
            total_neg += tf.reduce_sum(tf.cast(true_labels == 0, tf.int32)).numpy()
            total_pos += tf.reduce_sum(tf.cast(true_labels == 1, tf.int32)).numpy()
            num_samples += 1

        print(f'Epoch {epoch+1}/{epochs} - Training Loss: {running_loss / num_samples}')
        calculate_metrics(true_labels.numpy(), predicted.numpy(), num_correct_neg, total_neg, num_correct_pos, total_pos)

        # Validation phase
        running_loss_val = 0.0
        num_correct_neg_val = num_correct_pos_val = total_neg_val = total_pos_val = 0
        num_samples_val = 0

        for attributes, isic_id, patient_id, labels in tqdm(val_dataset):
            outputs = model(attributes, training=False)
            loss_val = loss_fn(labels, outputs)
            running_loss_val += loss_val.numpy()

            predicted_val = tf.argmax(outputs, axis=1)
            true_labels_val = tf.argmax(labels, axis=1)

            correct_neg_val = tf.reduce_sum(tf.cast((predicted_val == 0) & (true_labels_val == 0), tf.int32)).numpy()
            correct_pos_val = tf.reduce_sum(tf.cast((predicted_val == 1) & (true_labels_val == 1), tf.int32)).numpy()
            num_correct_neg_val += correct_neg_val
            num_correct_pos_val += correct_pos_val
            total_neg_val += tf.reduce_sum(tf.cast(true_labels_val == 0, tf.int32)).numpy()
            total_pos_val += tf.reduce_sum(tf.cast(true_labels_val == 1, tf.int32)).numpy()
            num_samples_val += 1

        val_loss = running_loss_val / num_samples_val
        print(f'Epoch {epoch+1}/{epochs} - Validation Loss: {val_loss}')
        calculate_metrics(true_labels_val.numpy(), predicted_val.numpy(), num_correct_neg_val, total_neg_val, num_correct_pos_val, total_pos_val)

        # Early stopping logic
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered")
                break

# Example usage:


In [12]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, losses, metrics

import numpy as np
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

class MyModel(tf.keras.Model):
    def __init__(self, num_attributes):
        super(MyModel, self).__init__()
        self.num_attributes = num_attributes  # Save this parameter for re-creation

        # Model for attributes
        self.attributes_model = tf.keras.Sequential([
            layers.Dense(64, activation='relu'),
            layers.Dense(32, activation='relu'),
            layers.Dense(2, activation='softmax')  # Output layer for binary classification
        ])

    def call(self, attributes):
        # Attributes model forward pass
        outputs = self.attributes_model(attributes)
        return outputs

    def get_config(self):
        # Return a dictionary of parameters to reconstruct the model
        config = super(MyModel, self).get_config()
        config.update({
            'num_attributes': self.num_attributes
        })
        return config

    @classmethod
    def from_config(cls, config):
        # Recreate the model from the configuration
        return cls(num_attributes=config['num_attributes'])


In [13]:
model = MyModel(num_attributes= 62)  # Define your model here
optimizer = tf.optimizers.Adam()
loss_fn = tf.losses.CategoricalCrossentropy()

mg_path = "/kaggle/input/training/npy_images"

train_dataset = ImgDatasetTF(val_stride=10, isValSet_bool=False,
                             img_path=None, ratio_int=1, is_train=True, train_df=train_df)
val_dataset = ImgDatasetTF(val_stride=10, isValSet_bool=True,
                           img_path=mg_path, is_train=True, train_df=train_df)


# Train and validation dataset created from ImgDatasetTF class
train_tf_dataset = train_dataset.to_tf_dataset(batch_size=32)
val_tf_dataset = val_dataset.to_tf_dataset(batch_size=32)

train_model(model, train_tf_dataset, val_tf_dataset, epochs=5, optimizer=optimizer, loss_fn=loss_fn, patience=3)

6250it [04:45, 21.90it/s]


Epoch 1/5 - Training Loss: 0.09819168377641588
TN=16, FP=0, FN=2, TP=14, Precision=1.00, Recall=0.88, F1=0.93
Accuracy: 97.07% (98456 of 100000 negative, 95693 of 100000 positive)


1254it [00:21, 58.35it/s]


Epoch 1/5 - Validation Loss: 0.05663769093195423
TN=0, FP=0, FN=0, TP=0, Precision=1.00, Recall=1.00, F1=1.00
Accuracy: 98.27% (32 of 60 negative, 39379 of 40046 positive)


6250it [04:42, 22.12it/s]


...

Epoch 4/5 - Validation Loss: 0.039431468885815314
TN=0, FP=0, FN=1, TP=9, Precision=1.00, Recall=0.90, F1=0.95
Accuracy: 99.06% (19 of 60 negative, 39710 of 40046 positive)


6250it [04:43, 22.04it/s]


Epoch 5/5 - Training Loss: 0.01291537980471563
TN=16, FP=0, FN=1, TP=15, Precision=1.00, Recall=0.94, F1=0.97
Accuracy: 99.68% (99875 of 100000 negative, 99493 of 100000 positive)


1254it [00:21, 57.41it/s]

Epoch 5/5 - Validation Loss: 0.04218070611868397
TN=0, FP=0, FN=1, TP=9, Precision=1.00, Recall=0.90, F1=0.95
Accuracy: 99.12% (14 of 60 negative, 39740 of 40046 positive)
Early stopping triggered


In [15]:
# Load your entire dataset
# dataset = ImgDatasetTF(val_stride=10, isValSet_bool=False, img_path=None, ratio_int=1, is_train=True, train_df=train_df)

In [ ]:
# from sklearn.model_selection import KFold


# kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# for fold, (train_idx, val_idx) in enumerate(kfold.split(dataset)):
#     print(f"Fold {fold+1}")

#     # Split the dataset into training and validation sets
#     train_data = np.array(dataset)[train_idx]
#     val_data = np.array(dataset)[val_idx]

#     # Convert to TensorFlow datasets
#     train_tf_dataset = to_tf_dataset(train_data, batch_size=32)
#     val_tf_dataset = to_tf_dataset(val_data, batch_size=32)

#     # Apply optimizations
#     train_tf_dataset = train_tf_dataset.shuffle(buffer_size=10000).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
#     val_tf_dataset = val_tf_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

#     # Train the model
#     train_model(model, train_tf_dataset, val_tf_dataset, epochs=5, optimizer=optimizer, loss_fn=loss_fn, patience=3)


Fold 1


In [14]:
model_json = model.to_json()
with open("model_skin2.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("/kaggle/working/model_skin2.weights.h5")

# Load the model architecture and weights separately
from tensorflow.keras.models import model_from_json

with open("/kaggle/working/model_skin2.json", "r") as json_file:
    loaded_model_json = json_file.read()
loaded_model = model_from_json(loaded_model_json, custom_objects={'MyModel': MyModel})
loaded_model.load_weights("/kaggle/working/model_skin2.weights.h5")

In [ ]:
# # Specify the directory where the model will be saved
# model_save_path = '/kaggle/working/model_skin3.h5'

# # Save the model
# model.save(model_save_path)
# print(f"Model saved at {model_save_path}")


In [15]:
import tensorflow as tf
import numpy as np
import copy

class TestDataset:
    def __init__(self, isic_id=None, img_path=None, is_train=None, test_df=None, transform=None):
        self.test_df = test_df
        self.is_train = is_train
        self.imgInfo_list = copy.copy(getCandidateInfoList(self.test_df, self.is_train))
        self.img_path = img_path
        self.transform = transform

        if isic_id:
            self.imgInfo_list = [x for x in self.imgInfo_list if x.isic_id == isic_id]

    def generator(self):
        for imgInfo_tup in self.imgInfo_list:
            attributes = np.array(imgInfo_tup.attributes, dtype=np.float32)
            yield imgInfo_tup.isic_id, imgInfo_tup.patient_id, attributes

    def to_tf_dataset(self):
        return tf.data.Dataset.from_generator(
            self.generator,
            output_signature=(
                tf.TensorSpec(shape=(), dtype=tf.string),  # ISIC ID
                tf.TensorSpec(shape=(), dtype=tf.string),  # Patient ID
                tf.TensorSpec(shape=(len(self.imgInfo_list[0].attributes),), dtype=tf.float32)  # Attributes
            )
        )





In [16]:
import numpy as np
import tensorflow as tf
from tqdm import tqdm
import gc

def predict(model, dataset, batch_size=32, device='cpu'):
    results = []
    
    device = '/GPU:0' if tf.config.list_physical_devices('GPU') else '/CPU:0'

    # Explicitly specify the device (GPU or CPU)
    with tf.device(device):
        for batch in tqdm(dataset.batch(batch_size)):
            isic_id, patient_id, attributes = batch
            
            # Decode byte strings to regular strings
            isic_id = [id.decode('utf-8') for id in isic_id.numpy()]

            # Move data to the specified device automatically based on model's device
            outputs = model(attributes, training=False)  # Inference mode

            # Apply softmax to the outputs to get probabilities
            probs = tf.nn.softmax(outputs, axis=1).numpy()

            # Extract malignant class probabilities (assuming class 1 is malignant)
            malignant_probs = probs[:, 1]

            # Round the probabilities to one decimal place
            malignant_probs = np.round(malignant_probs, 1)

            # Store the ISIC IDs and corresponding probabilities
            results.extend(zip(isic_id, malignant_probs))

            # Perform garbage collection
            gc.collect()

    return results




In [19]:
# Create an instance of the TestDataset
test_dataset = TestDataset(is_train=False, img_path=None, test_df=test_df).to_tf_dataset()


# Run the prediction function
predictions = predict(loaded_model, test_dataset, batch_size=32)


# predictions


1it [00:02,  2.14s/it]


In [20]:
df = pd.DataFrame(predictions, columns=['isic_id', 'target'])

In [21]:
df

,isic_id,target
0,ISIC_0015740,0.7
1,ISIC_0015729,0.7
2,ISIC_0015657,0.7


In [22]:
df.to_csv('submission.csv', index=False)